In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with LlamaIndex on Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-llamaindex-on-vertexai.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fintro-llamaindex-on-vertexai.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-llamaindex-on-vertexai.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-llamaindex-on-vertexai.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner) |

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.11

## Overview

LlamaIndex is a data framework for developing context-augmented large language model (LLM) applications. Context augmentation occurs when you apply an LLM to your data. This implements retrieval-augmented generation (RAG).

A common problem with LLMs is that they don't understand private knowledge, that is, your organization's data. With LlamaIndex on Vertex AI for RAG, you can enrich the LLM context with additional private information, because the model can reduce hallucination and answer questions more accurately.

By combining additional knowledge sources with the existing knowledge that LLMs have, a better context is provided. The improved context along with the query enhances the quality of the LLM's response.

For more information, refer to the official documentation and API Reference:

- https://cloud.google.com/vertex-ai/generative-ai/docs/llamaindex-on-vertexai
- https://cloud.google.com/vertex-ai/generative-ai/docs/model-reference/rag-api

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.
1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).
1. Enable the [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) and [Vertex AI Search and Conversation API](https://console.cloud.google.com/flows/enableapi?apiid=discoveryengine.googleapis.com).
1. If you want to use Grounding with Google Web Search results, your project must also be allowlisted for this feature while it is in the Private Preview stage.
1. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

### Installation

Install the following packages required to execute this notebook.

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

Restart the kernel after installing packages:

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Please wait until it is finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your Google Cloud account

If you are running this notebook on Google Colab, you will need to authenticate your environment. To do this, run the new cell below. This step is not required if you are using Vertex AI Workbench.

In [ ]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [21]:
PROJECT_ID = "document-ai-test-337818"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

# Set the project ID
!gcloud auth application-default set-quota-project {PROJECT_ID}
!gcloud config set project {PROJECT_ID}


Credentials saved to file: [/Users/holtskinner/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "document-ai-test-337818" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.
Updated property [core/project].


In [22]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

### Import libraries

In [23]:
from vertexai.preview.generative_models import (
    GenerativeModel,
    Tool,
)
from vertexai.preview import rag


from IPython.display import display, Markdown

Initialize the Gemini model from Vertex AI:

### Create a RAG Corpus

In [12]:
display_name = "Cymbal Cars Corpus"  # @param {type: "string"}
rag_corpus = rag.create_corpus(display_name=display_name)

### Import Files to the RagCorpus


In [39]:
paths = [
    "gs://github-repo/generative-ai/sample-apps/fixmycar/cymbal-starlight-2024.pdf"
]  # @param {type:"raw"}

response = rag.upload_file(
    "projects/908687846511/locations/us-central1/ragCorpora/4206925001917464576",
    "/Users/holtskinner/Downloads/cymbal-starlight-2024.pdf",
    display_name="cymbal-starlight-2024.pdf",
    description="Manual for 2024 Cymbal Starlight",
)


### Direct context retrieval

In [40]:
rag_corpus_id = (
    "projects/908687846511/locations/us-central1/ragCorpora/4206925001917464576"
)

In [44]:
query = "What is maximum cargo capacity of the 2024 Cymbal Starlight?"  # @param {type:"string"}

response = rag.retrieval_query(
    rag_resources=[
        rag.RagResource(
            rag_corpus=rag_corpus_id,
        )
    ],
    text=query,
    similarity_top_k=3
)

In [45]:
for context in response.contexts.contexts:
    print(context)

source_uri: "cymbal-starlight-2024.pdf"
text: "This light may illuminate for a variety of reasons, including:\r\nA low oil level\r\nA faulty oil pump\r\nA leak in the oil system\r\nCoolant temperature warning light: The coolant temperature warning light indicates that the\r\ncoolant temperature in the engine is high. This light may illuminate for a variety of reasons, including:\r\nA low coolant level\r\nA faulty thermostat\r\nA leak in the cooling system\r\nSafety Warning Lights\r\nThe following safety warning lights may illuminate in the instrument cluster of your Cymbal Starlight 2024:\r\nABS warning light: The ABS warning light indicates a problem with the vehicle\'s anti-lock braking\r\nsystem. This light may illuminate for a variety of reasons, including:\r\nA faulty ABS sensor\r\nA problem with the ABS control module\r\nA leak in the ABS hydraulic system\r\nTraction control warning light: The traction control warning light indicates a problem with the\r\nvehicle\'s traction cont

### Generative Response

In [49]:
rag_model = GenerativeModel(
    model_name="gemini-1.5-pro-preview-0409",
    tools=[
        Tool.from_retrieval(
            retrieval=rag.Retrieval(
                source=rag.VertexRagStore(
                    rag_corpora=[rag_corpus_id],
                    similarity_top_k=3,
                    vector_distance_threshold=0.5,
                ),
            ),
        )
    ],
    system_instruction=[
        "You are a helpful car manual chatbot. Answer the car owner's question about their car.",
    ],
)

chat = rag_model.start_chat()

response = chat.send_message(query)

print(response.text)
# Generate response
# response = rag_model.generate_content(query)
# print(response.text)

The 2024 Cymbal Starlight has a cargo capacity of 13.5 cubic feet. 

